# Python for Data Analysis Workbook 6 (Time Series)

### Preliminaries

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

#### Note on Time-Series

Time-series data may either be *fixed frequency* or *irregular* <br>

How you mark and refer to the time series data depends on the application and you may have one of the following: <br>

1) ***Timestamps*** or specific instants in time <br>
2) ***Periods*** that are already delineated like month of January, year 2010, etc. <br>
3) ***Intervals*** of time that are indicated by a start/end timestamp <br>
4) ***Elapsed Time*** where each timestamp is a measure of time relative to a particular start time

### Date and Time Data Types and Tools

In [12]:
# Python standard library includes data types for date and time data as well as calendar-related functionality

print(datetime.now())
now = datetime.now()

print([now.year, now.month, now.day])
print('\n')
# Can also calculate temporal difference using timedelta

delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
print(delta)
print('\n')
print([delta.days, delta.seconds])

2020-07-02 22:55:14.420593
[2020, 7, 2]


926 days, 15:45:00


[926, 56700]


In [15]:
# Can use timedelta to shift a series 

from datetime import timedelta

start = datetime(2011, 1, 7)

start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

#### Converting between Strings and Datetime

In [17]:
# String --> DateTime

stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [19]:
# Datetime --> String

stamp.strftime('%Y-%m-%d') # Need to provide a format based on codes

'2011-01-03'

**Codes**

%Y - four-digit year <br>
%y - two-digit year <br>
%m - two-digit month <br>
%d - two-digit day <br>
%H - Hour, military time <br>
%I - Hour, regular time <br>
%M - two-digit minute <br>
%s - second (60 & 61 account for leap seconds) <br>
%w - weekday as integer (starting with Sunday = 0 <br>
%W - week number <br>
%z - UTC time <br>
%a - abbreviated weekday name <br>
%A - full weekday name <br>
%b - abbreviated month name <br>
%B - full month name <br>
%c - full date and time <br>
%p - local equivalent of am/pm <br>
%x - locale-appropriate formatted date <br>
%X - locale-appropriate formatted time <br>
%F - shortcut for %Y-%m-%d <br>
%D - shortcut for %m/%d/%y <br>

In [28]:
# String --> Datetime (using strptime())

value = '2011-01-03'
print(datetime.strptime(value, '%Y-%m-%d'))
print('\n')

# Can also just use the parse method from dateutil library, which can parse most human-readable date represenations

from dateutil.parser import parse
print(parse(value))
print('\n')
print(parse('6/12/2011', dayfirst = True))

2011-01-03 00:00:00


2011-01-03 00:00:00


2011-12-06 00:00:00


In [31]:
# Another way to convert to datetime, use pandas to_datetime method

datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

### Time Series Basics

In [38]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), 
         datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index = dates)
print(ts)

ts.index

2011-01-02    0.145210
2011-01-05   -0.323280
2011-01-07    2.043052
2011-01-08    1.159492
2011-01-10    0.803376
2011-01-12   -0.525725
dtype: float64


DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [41]:
ts + ts[::2] # Selects every second element in the Timeseries

2011-01-02    0.290419
2011-01-05         NaN
2011-01-07    4.086105
2011-01-08         NaN
2011-01-10    1.606752
2011-01-12         NaN
dtype: float64

In [44]:
# Returns a Timestamp object, which can be substituted anywhere for a datetime but also stores frequency info

ts.index[0] 

Timestamp('2011-01-02 00:00:00')

In [45]:
# Indexing works like any other pandas.Series and also can parse most date-like strings

ts['1/2/11']

0.1452095521005706

In [49]:
longer_ts = pd.Series(np.random.randn(1000), index = pd.date_range('1/1/2000', periods = 1000))
print(longer_ts)

longer_ts['2001']

2000-01-01   -1.606889
2000-01-02   -0.706294
2000-01-03   -0.022372
2000-01-04   -0.078844
2000-01-05   -1.884789
                ...   
2002-09-22    1.117063
2002-09-23   -0.721747
2002-09-24   -0.956305
2002-09-25    0.229228
2002-09-26    0.365482
Freq: D, Length: 1000, dtype: float64


2001-01-01   -0.466466
2001-01-02   -0.255349
2001-01-03    0.147402
2001-01-04   -1.517411
2001-01-05   -0.231846
                ...   
2001-12-27   -1.300224
2001-12-28    1.610084
2001-12-29   -0.691538
2001-12-30    1.291405
2001-12-31    0.193227
Freq: D, Length: 365, dtype: float64

In [55]:
# Can also slice

longer_ts['2001-12-27':]

2001-12-27   -1.300224
2001-12-28    1.610084
2001-12-29   -0.691538
2001-12-30    1.291405
2001-12-31    0.193227
                ...   
2002-09-22    1.117063
2002-09-23   -0.721747
2002-09-24   -0.956305
2002-09-25    0.229228
2002-09-26    0.365482
Freq: D, Length: 274, dtype: float64

In [57]:
# Another way to select a date range is to use the truncate method

ts.truncate(before = '1/3/2011', after = '1/9/2011')

2011-01-05   -0.323280
2011-01-07    2.043052
2011-01-08    1.159492
dtype: float64

#### Time Series with Duplicate Indices

In [62]:
dup_ts = pd.Series(np.arange(5), 
                   index = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000']))

print(dup_ts)
dup_ts.is_unique


2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64


True

In [64]:
# Can aggregate down using groupby

dup_ts.groupby(level = 0).count() # groupby index 

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

### Date Ranges, Shifting, and Frequencies

In [77]:
# Generating a date range

index = pd.date_range('2012-04-01', '2012-06-01')
print(pd.date_range(start = '2012-04-01', periods = 62)) # equivalent
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [79]:
# Can pass in custom frequencies

pd.date_range('2000-01-01', '2000-12-01', freq = 'BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [81]:
# Can also normalize time data by setting the normalize argument to True

pd.date_range('2012-05-02 12:56:31', periods = 5, normalize = True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

In [84]:
# Can also create custom frequencies by putting an integer beofre the base frequency

print(pd.date_range('2000-01-01', '2000-01-02', freq = '4h'))

pd.date_range('2000-01-01', '2000-01-02', freq = '1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00'],
              dtype='datetime64[ns]', freq='4H')


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00',
               '2000-01-01 15:00:00', '2000-01-01 16:30:00',
               '2000-01-01 18:00:00', '2000-01-01 19:30:00',
               '2000-01-01 21:00:00', '2000-01-01 22:30:00',
               '2000-01-02 00:00:00'],
              dtype='datetime64[ns]', freq='90T')

In [86]:
# One useful frequency class is week of month

rng = pd.date_range('2012-01-01', '2012-09-01', freq = 'WOM-3FRI')
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

In [92]:
# Can easily shift data within a date range forward or backwards using the shift() method

ts = pd.Series(np.abs(np.random.randn(4)), index = pd.date_range('1/1/2000', periods = 4, freq = 'M'))
print(ts)

ts.shift(2)
# ts.shift(-2)

2000-01-31    0.220500
2000-02-29    0.646986
2000-03-31    0.085404
2000-04-30    0.508564
Freq: M, dtype: float64


2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.220500
2000-04-30    0.646986
Freq: M, dtype: float64

In [93]:
# Common use of shift is to calculate percent changes

ts / ts.shift(1) - 1

2000-01-31         NaN
2000-02-29    1.934173
2000-03-31   -0.867997
2000-04-30    4.954780
Freq: M, dtype: float64

In [94]:
# Naive shift only does the data, but passing in a frequency enables the index to shift as well

ts.shift(2, freq = 'M')

2000-03-31    0.220500
2000-04-30    0.646986
2000-05-31    0.085404
2000-06-30    0.508564
Freq: M, dtype: float64

### Time-zone handling

In [97]:
import pytz # special library for handling time zones

In [98]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [99]:
# get timezone objet using .timezone
pytz.timezone('America/New_York')

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [100]:
# Date ranges can be generated with an explicit timezone set (default is none)
pd.date_range('3/9/2012 9:30', periods = 10, freq = 'D', tz = 'UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [102]:
# Can go from naive to localize using the localize() method

ts_utc = ts.tz_localize('UTC')
ts_utc

2000-01-31 00:00:00+00:00    0.220500
2000-02-29 00:00:00+00:00    0.646986
2000-03-31 00:00:00+00:00    0.085404
2000-04-30 00:00:00+00:00    0.508564
Freq: M, dtype: float64

In [103]:
# Can be converted with the tz_convert() method

ts_utc.tz_convert('America/New_York')

2000-01-30 19:00:00-05:00    0.220500
2000-02-28 19:00:00-05:00    0.646986
2000-03-30 19:00:00-05:00    0.085404
2000-04-29 20:00:00-04:00    0.508564
Freq: M, dtype: float64

In [106]:
# In addition to date ranges, individual timestamps can be made time-zone aware

stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
print(stamp_utc)

# Or pass a time zone directly into the Timestamp on creation

pd.Timestamp('2011-03-12 04:00', tz = 'Europe/Moscow')

2011-03-12 04:00:00+00:00


Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [107]:
# all timestamp objects store the Unix timestamp value 

stamp_utc.value

1299902400000000000

*Note, joining two different time zone series results in all being converted to UTC*

### Periods and Period Arithmetic

In [96]:
# Periods represent timespans like days, months, or quarters. Period class requires an integer and frequency

p = pd.Period(2007, freq = 'A-DEC')
p

Period('2007', 'A-DEC')